In [ ]:
speeds = cnt_lhm[(cnt_lhm['metric']=='speed') & (cnt_lhm['vehicle_class']=='PC')].groupby(['road_link_id', 'date'])[[str(i) for i in list(range(1,25))]].median(numeric_only=True)
speeds = speeds.loc[(speeds!=0).any(axis=1)]
speeds = speeds.dropna()
volumes = cnt_lhm[(cnt_lhm['metric']=='volume') & (cnt_lhm['vehicle_class']=='SUM')].groupby(['road_link_id', 'date'])[[str(i) for i in list(range(1,25))]].sum(numeric_only=True)

In [ ]:
road_links = list(speeds.index.get_level_values(0).unique())
i =0

In [ ]:
def get_hour_value(df, road_link_id, date, hour): 
    
    try:
        df_day = df.loc[road_link_id, date]
        return df_day.iloc[hour]
    except: 
        return np.nan

data_combined = pd.DataFrame(index = pd.date_range(start= '2019-01-01 00:00:00',
                                                end = '2019-12-31 23:00:00',
                                                freq= '1h'))

data_combined['date'] = data_combined.index.date
data_combined['hour'] = data_combined.index.hour

road_link_id = 67624481#road_links[i]
road_link_id = road_links[i]
i+=1
data_combined['speed'] = data_combined.apply(lambda row: get_hour_value(speeds,
                                                                        road_link_id,
                                                                        row['date'].strftime('%Y-%m-%d'),
                                                                        row['hour']), axis=1)
data_combined['volume'] = data_combined.apply(lambda row: get_hour_value(volumes,
                                                                        road_link_id,
                                                                        row['date'].strftime('%Y-%m-%d'),
                                                                        row['hour']), axis=1)

data_combined = data_combined.dropna()

data_combined['density'] = data_combined['volume'] / data_combined['speed']

plt.scatter(x =data_combined['volume'], y = data_combined['speed'], s = 2)

capacity = visum_links[visum_links['road_link_id'] == road_link_id]['hour_capacity'].iloc[0]
road_type = visum_links[visum_links['road_link_id'] == road_link_id]['road_type'].iloc[0]
plt.axvline(capacity, color = 'red')#

plt.xlabel('volume')
plt.ylabel('speed')
plt.title(f'roadlink: {road_link_id}, {road_type}')